# Prompt

First, you learned about the importance of transforming your data into a stationary series. You tested for stationarity using the adfuller function and interpreted the value of the hypothesis test. If the data was not stationary, you learned how to apply strategies, such as differencing and logarithmic transformations, to achieve stationarity.

Once the series was stationary, building an ARMA model involved using autocorrelation and partial autocorrelation plots to determine the appropriate p and q parameters of the model.

This activity asks you to identify a time series of interest to you and build an ARMA model to construct a basic forecast for the series and analyze the error. You might also consider building models with different p and q parameters because, while ACF and PACF plots are helpful, they provide rough ideas of the appropriate parameters, and it is usually good practice to perform a simple grid search on these. 

Please find a time series dataset using any resource you would like, and present your model and findings to the class.  

If you have trouble locating a dataset or would prefer a suggestion, try the [Rossmann Store Sales](https://www.kaggle.com/c/rossmann-store-sales) from Kaggle. This dataset involves forecasting retail store sales for a major drug store.

# Imports

In [ ]:
import warnings
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

from statsmodels.tsa.seasonal import seasonal_decompose, _extrapolate_trend
from statsmodels.tsa.filters.filtertools import convolution_filter
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.tsa.forecasting.stl import STLForecast
from sklearn.metrics import root_mean_squared_error, mean_absolute_error

In [ ]:
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
mpl.rcParams.update({"axes.grid": True})

# Data Load

Looking at snowfall per month for Mammoth Mountain, from 1969 to the present, as obtained from  
- https://www.mammothmountain.com/on-the-mountain/snowfall-history

See [pre-processing notebook](./mammoth_snowfall_datagen.ipynb) for how I generated this table from the Mammoth website data

In [ ]:
df = pd.read_csv("./data/mammoth_mountain_snowfall_melted_table.csv")
df.head()

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date").drop(columns="Month")
df.head()

# All Data

## Plot

In [ ]:
y_all = df["Snow Fall"]

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(y_all, color="blue", lw=2)
plt.title("Monthly Snow Fall at Mammoth Mountain")
plt.xlabel("Time")
plt.ylabel("Monthly Snow Fall (in)")
plt.show()

## Not Stationary

In [ ]:
plot_acf(y_all, lags=50)
plt.xlabel("Lag")
plt.ylabel("Corr")
plt.show()

In [ ]:
display(adfuller(y_all))

# First Difference

## Plot

In [ ]:
y_diff = df["Snow Fall"].diff().dropna()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(y_diff, color="blue", lw=2)
plt.title("First Difference of Monthly Snow Fall at Mammoth Mountain")
plt.xlabel("Time")
plt.ylabel("Month-to-Month Difference in Snow Fall (in)")
# plt.legend(loc="upper left")
plt.show()

## Somewhat Stationary

In [ ]:
(fig, ax) = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))
plot_acf(y_diff, lags=50, ax=ax[0])
plt.xlabel("Lag")
plt.ylabel("Corr")
plot_pacf(y_diff, lags=50, ax=ax[1])
plt.xlabel("Lag")
plt.ylabel("Corr")
plt.show()

In [ ]:
display(adfuller(y_diff))

# Second Difference

In [ ]:
y_diff2 = y_diff.diff().dropna()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(y_diff2, color="blue", lw=2)
plt.title("Second Difference of Monthly Snow Fall at Mammoth Mountain")
plt.xlabel("Time")
plt.ylabel("Month-to-Month Difference in Snow Fall (in)")
# plt.legend(loc="upper left")
plt.show()

## More Stationary - Use AR(p=4) and MA(q=1)

So choose to model 2nd difference

AR(p)
- PACF
- p = 4  

MA(q)
- ACF
- q = 1  

In [ ]:
(fig, ax) = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))
plot_acf(y_diff2, lags=10, ax=ax[0])
plt.xlabel("Lag")
plt.ylabel("Corr")
plot_pacf(y_diff2, lags=10, ax=ax[1])
plt.xlabel("Lag")
plt.ylabel("Corr")
plt.show()

In [ ]:
display(adfuller(y_diff2))

# Train / Test Split on y_diff2

In [ ]:
y_target = y_diff2

In [ ]:
n_months_future = 16
y_history = y_target[: len(y_target) - n_months_future]
y_future = y_target[len(y_target) - n_months_future :]

# Fit with ARIMA

In [ ]:
order_p_d_q = (2, 0, 2)
estimator = ARIMA(y_history, order=order_p_d_q).fit()
y_fit = estimator.predict()
y_forecast = estimator.forecast(steps=len(y_future))
mae_fit = mean_absolute_error(y_history, y_fit)
rmse_fit = root_mean_squared_error(y_history, y_fit)
plt.plot(y_history, color="black")
plt.plot(y_fit, color="blue")
plt.title("Fit Errors MAE = %.2f, RMSE = %.2f" % (mae_fit, rmse_fit))

In [ ]:
mae_fit = mean_absolute_error(y_history, y_fit)
rmse_fit = root_mean_squared_error(y_history, y_fit)
mae_forecast = mean_absolute_error(y_future, y_forecast)
rmse_forecast = root_mean_squared_error(y_future, y_forecast)
[mae_fit, rmse_fit, mae_forecast, rmse_forecast]

In [ ]:
nplot = 10*12
plt.figure(figsize=(16, 4))
plt.plot(y_history[-nplot:], color="black", label="Observed")
plt.plot(
    y_fit[-nplot:],
    color="blue",
    label="Fit (MAE, RMSE) = %.2f, %.2f" % (mae_fit, rmse_fit),
)
# plt.plot(
#     y_forecast[-nplot:],
#     color="blue",
#     label="Sarima Forecast (MAE, RMSE) = %.2f, %.2f"
#     % (mae_forecast, rmse_forecast),
# )
plt.title("Monthly Snow Fall at Mammoth Mountain")
plt.xlabel("Year")
plt.ylabel("Monthly Snow Fall (inches)")
plt.legend(loc="upper left")
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(estimator.resid, color="black", lw=2, label="Fit Residual")
plt.title("Model Residue")
plt.xlabel("Year")
plt.ylabel("Fit Error (inches)")
plt.legend(loc="upper left")
plt.show()

In [ ]:
(fig, ax) = plt.subplots(nrows=1, ncols=1, figsize=(16, 8))
ax.plot(y_future, color="black", lw=2, label="Future")
ax.plot(y_forecast, color="blue", lw=2, label="Sarima Forecast")
ax.set_ylabel("Monthly Snowfall (inches)")

# ax.plot(
#     y_future - y_forecast,
#     color="red",
#     lw=2,
#     label="Sarima Forecast Errors (MAE, RMSE) = %.2f, %.2f"
#     % (mae_forecast, rmse_forecast),
# )

ax.set_xlabel("Year")
ax.legend(loc="upper left")
plt.title("Monthly Snowfall, Future and Forecasted")
plt.show()